# December 2017: Advent of Code Solutions

Peter Norvig

I'm doing the [Advent of Code](https://adventofcode.com) puzzles, just like [last year](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb). But this time, I won't write up my interpretation of each day's the puzzle description; you'll have to follow the links in the section headers (e.g. **[Day 1](https://adventofcode.com/2017/day/1)**) to read those. I just show my solutions.

# Day 0: Imports and Utility Functions

I might need these:

In [1]:
# Python 3.x Utility Functions

import re
import numpy as np
import math
import random
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque, abc, OrderedDict
from functools   import lru_cache
from itertools   import (permutations, combinations, chain, cycle, product, islice, 
                         takewhile, zip_longest, count as count_from)
from heapq       import heappop, heappush

identity = lambda x: x
letters  = 'abcdefghijklmnopqrstuvwxyz'

cat = ''.join

Ø   = frozenset() # Empty set
inf = float('inf')
BIG = 10 ** 999

################ Functions for Input, Parsing

def Input(day, year=2017):
    "Open this day's input file."
    return open('data/advent{}/input{}.txt'.format(year, day))
    
def array(lines):
    "Parse an iterable of str lines into a 2-D array. If `lines` is a str, do splitlines."
    if isinstance(lines, str): lines = lines.splitlines()
    return mapt(vector, lines)

def vector(line):
    "Parse a str into a tuple of atoms (numbers or str tokens)."
    return mapt(atom, line.split())

def atom(token):
    "Parse a str token into a number, or leave it as a str."
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return token

################ Functions on Iterables

def first(iterable, default=None): return next(iter(iterable), default)

def first_true(iterable, pred=None, default=None):
    """Returns the first true value in the iterable.
    If no true value is found, returns *default*
    If *pred* is not None, returns the first item
    for which pred(item) is true."""
    # first_true([a,b,c], default=x) --> a or b or c or x
    # first_true([a,b], fn, x) --> a if fn(a) else b if fn(b) else x
    return next(filter(pred, iterable), default)

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

def upto(iterable, maxval):
    "From a monotonically increasing iterable, generate all the values <= maxval."
    # Why <= maxval rather than < maxval? In part because that's how Ruby's upto does it.
    return takewhile(lambda x: x <= maxval, iterable)

def groupby(iterable, key=identity):
    "Return a dict of {key(item): [items...]} grouping all items in iterable by keys."
    groups = defaultdict(list)
    for item in iterable:
        groups[key(item)].append(item)
    return groups

def grouper(iterable, n, fillvalue=None):
    """Collect data into fixed-length chunks:
    grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"""
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

def overlapping(iterable, n):
    """Generate all (overlapping) n-element subsequences of iterable.
    overlapping('ABCDEFG', 3) --> ABC BCD CDE DEF EFG"""
    if isinstance(iterable, abc.Sequence):
        yield from (iterable[i:i+n] for i in range(len(iterable) + 1 - n))
    else:
        result = deque(maxlen=n)
        for x in iterable:
            result.append(x)
            if len(result) == n:
                yield tuple(result)
                
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    return overlapping(iterable, 2)

def sequence(iterable, type=tuple):
    "Coerce iterable to sequence: leave it alone if it is already a sequence, else make it of type."
    return iterable if isinstance(iterable, abc.Sequence) else type(iterable)

def join(iterable, sep=''):
    "Join the items in iterable, converting each to a string first."
    return sep.join(map(str, iterable))
                
def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c
            
def quantify(iterable, pred=bool):
    "Count how many times the predicate is true."
    return sum(map(pred, iterable))

def shuffled(iterable):
    "Create a new list out of iterable, and shuffle it."
    new = list(iterable)
    random.shuffle(new)
    return new
    
flatten = chain.from_iterable
            
class Set(frozenset):
    "A frozenset, but with a prettier printer."
    def __repr__(self): return '{' + join(sorted(self), ', ') + '}'
    
def canon(items, typ=None):
    "Canonicalize these order-independent items into a hashable canonical form."
    typ = typ or (cat if isinstance(items, str) else tuple)
    return typ(sorted(items))

def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))
            
################ Math Functions
            
def transpose(matrix): return tuple(zip(*matrix))

def isqrt(n):
    "Integer square root (rounds down)."
    return int(n ** 0.5)

def ints(start, end):
    "The integers from start to end, inclusive: range(start, end+1)"
    return range(start, end + 1)

def floats(start, end, step=1.0):
    "Yields from start to end (inclusive), by increments of step."
    m = (1.0 if step >= 0 else -1.0)
    while start * m <= end * m:
        yield start
        start += step
        
def multiply(numbers):
    "Multiply all the numbers together."
    result = 1
    for n in numbers:
        result *= n
    return result

################ 2-D points implemented using (x, y) tuples

def X(point): x, y = point; return x
def Y(point): x, y = point; return y

origin = (0, 0)
UP, DOWN, LEFT, RIGHT = (0, 1), (0, -1), (-1, 0), (1, 0)

def neighbors4(point): 
    "The four neighboring squares."
    x, y = point
    return (          (x, y-1),
            (x-1, y),           (x+1, y), 
                      (x, y+1))

def neighbors8(point): 
    "The eight neighboring squares."
    x, y = point 
    return ((x-1, y-1), (x, y-1), (x+1, y-1),
            (x-1, y),             (x+1, y),
            (x-1, y+1), (x, y+1), (x+1, y+1))

def cityblock_distance(p, q=origin): 
    "Manhatten distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def distance(p, q=origin): 
    "Hypotenuse distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

################ Debugging 

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def grep(pattern, iterable):
    "Print lines from iterable that match pattern."
    for line in iterable:
        if re.search(pattern, line):
            print(line)

################ A* and Breadth-First Search (tracking states, not actions)

def always(value): return (lambda *args: value)

def Astar(start, moves_func, h_func, cost_func=always(1)):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    Path      = lambda s: ([] if (s is None) else Path(previous[s]) + [s])
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(s)
        for s2 in moves_func(s):
            g = path_cost[s] + cost_func(s, s2)
            if s2 not in path_cost or g < path_cost[s2]:
                heappush(frontier, (g + h_func(s2), s2))
                path_cost[s2] = g
                previous[s2] = s

def bfs(start, moves_func, goals):
    "Breadth-first search"
    goal_func = (goals if callable(goals) else lambda s: s in goals)
    return Astar(start, moves_func, lambda s: (0 if goal_func(s) else 1))

In [2]:
def tests():
    # Functions for Input, Parsing
    assert array('''1 2 3
                    4 5 6''') == ((1, 2, 3), 
                                  (4, 5, 6))
    assert vector('testing 1 2 3.') == ('testing', 1, 2, 3.0)
    
    # Functions on Iterables
    assert first('abc') == first(['a', 'b', 'c']) == 'a'
    assert first_true([0, None, False, {}, 42, 43]) == 42
    assert nth('abc', 1) == nth(iter('abc'), 1) == 'b'
    assert cat(upto('abcdef', 'd')) == 'abcd'
    assert cat(['do', 'g']) == 'dog'
    assert groupby([-3, -2, -1, 1, 2], abs) == {1: [-1, 1], 2: [-2, 2], 3: [-3]}
    assert list(grouper(range(8), 3)) == [(0, 1, 2), (3, 4, 5), (6, 7, None)]
    assert list(overlapping((0, 1, 2, 3, 4), 3)) == [(0, 1, 2), (1, 2, 3), (2, 3, 4)]
    assert list(overlapping('abcdefg', 4)) == ['abcd', 'bcde', 'cdef', 'defg']  
    assert list(pairwise((0, 1, 2, 3, 4))) == [(0, 1), (1, 2), (2, 3), (3, 4)]
    assert sequence('seq') == 'seq'
    assert sequence((i**2 for i in range(5))) == (0, 1, 4, 9, 16)
    assert join(range(5)) == '01234'
    assert join(range(5), ', ') == '0, 1, 2, 3, 4'
    assert multiply([1, 2, 3, 4]) == 24
    assert transpose(((1, 2, 3), (4, 5, 6))) == ((1, 4), (2, 5), (3, 6))
    assert isqrt(9) == 3 == isqrt(10)
    assert ints(1, 100) == range(1, 101)
    assert identity('anything') == 'anything'
    assert set(powerset({1, 2, 3})) == {(), (1,), (1, 2), (1, 2, 3), (1, 3), (2,), (2, 3), (3,)}
    assert quantify(['testing', 1, 2, 3, int, len], callable) == 2 # int and len are callable
    assert quantify([0, False, None, '', [], (), {}, 42]) == 1  # Only 42 is truish
    assert set(shuffled('abc')) == set('abc')
    assert canon('abecedarian') == 'aaabcdeeinr'
    assert canon([9, 1, 4]) == canon({1, 4, 9}) == (1, 4, 9)
    assert mapt(math.sqrt, [1, 9, 4]) == (1, 3, 2)
    
    # Math
    assert transpose([(1, 2, 3), (4, 5, 6)]) == ((1, 4), (2, 5), (3, 6))
    assert isqrt(10) == isqrt(9) == 3
    assert ints(1, 5) == range(1, 6)
    assert list(floats(1, 5)) == [1., 2., 3., 4., 5.]
    assert multiply(ints(1, 10)) == math.factorial(10) == 3628800
    
    # 2-D points
    P = (3, 4)
    assert X(P) == 3 and Y(P) == 4
    assert cityblock_distance(P) == cityblock_distance(P, origin) == 7
    assert distance(P) == distance(P, origin) == 5
    
    # Search
    assert Astar((4, 4), neighbors8, distance) == [(4, 4), (3, 3), (2, 2), (1, 1), (0, 0)]
    assert bfs((4, 4), neighbors8, {origin}) == [(4, 4), (3, 3), (2, 2), (1, 1), (0, 0)]
    forty2 = always(42)
    assert forty2() == forty2('?') == forty2(4, 2) == 42

    return 'pass'

tests()

'pass'

# [Day 1](https://adventofcode.com/2017/day/1): Inverse Captcha


In [3]:
digits = mapt(int, '3294199471327195994824832197564859876682638188889768298894243832665654681412886862234525991553276578641265589959178414218389329361496673991614673626344552179413995562266818138372393213966143124914469397692587251112663217862879233226763533911128893354536353213847122251463857894159819828724827969576432191847787772732881266875469721189331882228146576832921314638221317393256471998598117289632684663355273845983933845721713497811766995367795857965222183668765517454263354111134841334631345111596131682726196574763165187889337599583345634413436165539744188866156771585647718555182529936669683581662398618765391487164715724849894563314426959348119286955144439452731762666568741612153254469131724137699832984728937865956711925592628456617133695259554548719328229938621332325125972547181236812263887375866231118312954369432937359357266467383318326239572877314765121844831126178173988799765218913178825966268816476559792947359956859989228917136267178571776316345292573489873792149646548747995389669692188457724414468727192819919448275922166321158141365237545222633688372891451842434458527698774342111482498999383831492577615154591278719656798277377363284379468757998373193231795767644654155432692988651312845433511879457921638934877557575241394363721667237778962455961493559848522582413748218971212486373232795878362964873855994697149692824917183375545192119453587398199912564474614219929345185468661129966379693813498542474732198176496694746111576925715493967296487258237854152382365579876894391815759815373319159213475555251488754279888245492373595471189191353244684697662848376529881512529221627313527441221459672786923145165989611223372241149929436247374818467481641931872972582295425936998535194423916544367799522276914445231582272368388831834437562752119325286474352863554693373718848649568451797751926315617575295381964426843625282819524747119726872193569785611959896776143539915299968276374712996485367853494734376257511273443736433464496287219615697341973131715166768916149828396454638596713572963686159214116763')
N = len(digits)

In [4]:
sum(digits[i] 
    for i in range(N) 
    if digits[i] == digits[i - 1])

1158

**Part Two**:

In [5]:
sum(digits[i] 
    for i in range(N) 
    if digits[i] == digits[i - N//2])

1132

This was an easy warmup puzzle. 

# [Day 2](https://adventofcode.com/2017/day/2): Corruption Checksum


In [6]:
rows = array('''790	99	345	1080	32	143	1085	984	553	98	123	97	197	886	125	947
302	463	59	58	55	87	508	54	472	63	469	419	424	331	337	72
899	962	77	1127	62	530	78	880	129	1014	93	148	239	288	357	424
2417	2755	254	3886	5336	3655	5798	3273	5016	178	270	6511	223	5391	1342	2377
68	3002	3307	166	275	1989	1611	364	157	144	3771	1267	3188	3149	156	3454
1088	1261	21	1063	1173	278	1164	207	237	1230	1185	431	232	660	195	1246
49	1100	136	1491	647	1486	112	1278	53	1564	1147	1068	809	1638	138	117
158	3216	1972	2646	3181	785	2937	365	611	1977	1199	2972	201	2432	186	160
244	86	61	38	58	71	243	52	245	264	209	265	308	80	126	129
1317	792	74	111	1721	252	1082	1881	1349	94	891	1458	331	1691	89	1724
3798	202	3140	3468	1486	2073	3872	3190	3481	3760	2876	182	2772	226	3753	188
2272	6876	6759	218	272	4095	4712	6244	4889	2037	234	223	6858	3499	2358	439
792	230	886	824	762	895	99	799	94	110	747	635	91	406	89	157
2074	237	1668	1961	170	2292	2079	1371	1909	221	2039	1022	193	2195	1395	2123
8447	203	1806	6777	278	2850	1232	6369	398	235	212	992	7520	7304	7852	520
3928	107	3406	123	2111	2749	223	125	134	146	3875	1357	508	1534	4002	4417''')

In [7]:
sum(abs(max(row) - min(row)) for row in rows)

46402

**Part Two**:

In [8]:
def evendiv(row): 
    return first(a // b for a in row for b in row if a > b and a // b == a / b)

sum(map(evendiv, rows))

265

This day was also very easy. In Part One, I was slowed down by a typo: I had `"="` instead of `"-"` in `"max(row) - min(row)"`. I was confused by Python's misleading error message, which said `"SyntaxError: keyword can't be an expression"`. Later on, Alex Martelli explained to me that the message meant that in `abs(max(row)=...)` it thought that `max(row)` was a keyword argument to `abs`, as in `abs(x=-1)`.

In Part Two, note that to check that `a/b` is an exact integer, I used `a // b == a / b`, which I think is more clear and less error-prone than the expression one would typically use here, `a % b == 0`, which requires you to think about two things: division and the modulus operator (is it `a % b` or `b % a`?).

# [Day 3](https://adventofcode.com/2017/day/3): Spiral Memory

In [9]:
N = 277678

This one takes some thinking, not just fast typing. I analyzed the problem as having three parts:
- Generate a spiral
- Find the Nth square on the spiral. 
- Find the distance from that square to the center.

I suspect many people will do all three of these in one function. That's probably the best way to get the answer quickly, but I'd rather be clear than quick, so I'll factor out each part, according to the single responsibility principle. My function `spiral()` will generate the coordinates of squares on an infinite spiral, in order, going out from the center square, `(0, 0)`.

How to make a spiral? My analysis is that, after the center square, the spiral goes 1 square right, then 1 square up, then 2 square left, then 2 square down, to complete one revolution; the next revolution starts with 3 square going up, and so on. I'll call each of these a `leg`, so `spiral` consists of four calls to `leg`, with increments to the `length` after every two legs. 

One thing is less clear than I would like: the variable `square` is modified by the function `leg` (in other words, it is an in/out parameter). A small test confirms that this matches the puzzle description:

In [10]:
def spiral():
    "Yield the (x, y) coordinates of successive points in an infinite spiral."
    length = 1
    square = [0, 0]
    yield tuple(square)
    while True:
        yield from leg(square, length, RIGHT)
        yield from leg(square, length, UP)
        length += 1
        yield from leg(square, length, LEFT)
        yield from leg(square, length, DOWN)
        length += 1  
        
def leg(square, length, delta):
    "Complete one leg of given length, mutating `square` and yielding a copy at each step."
    for _ in range(length):
        square[:] = (X(square) + X(delta), Y(square) + Y(delta))
        yield tuple(square)  
        
list(islice(spiral(), 10))

[(0, 0),
 (1, 0),
 (1, 1),
 (0, 1),
 (-1, 1),
 (-1, 0),
 (-1, -1),
 (0, -1),
 (1, -1),
 (2, -1)]

Now we can find the `N`th square. As this is Python, indexes start at 0, whereas the problem starts at 1, so I have to subtract 1. Then I can find the distance to the origin:

In [11]:
nth(spiral(), N - 1)

(212, -263)

In [12]:
cityblock_distance(_)

475

That's the right answer. I was slow arriving at it because I forgot the second `length += 1` in `spiral` and it took a while  to debug. (I had the right analysis, but just left out a line of code.)

For **Part Two** I can re-use my `spiral` generator, yay! Here's a function to sum the neighboring squares (I can use my `neighbors8` function, yay!):

In [13]:
def spiralsums():
    "Yield the values of a spiral where each square has the sum of the 8 neighbors."
    value = defaultdict(int)
    for p in spiral():
        value[p] = sum(value[q] for q in neighbors8(p)) or 1
        yield value[p]

In [14]:
list(islice(spiralsums(), 12))

[1, 1, 2, 4, 5, 10, 11, 23, 25, 26, 54, 57]

Looks good, so let's get the answer:

In [15]:
first(x for x in spiralsums() if x > N)

279138

# [Day 4](https://adventofcode.com/2017/day/4): High-Entropy Passphrases

This is the first time I will have to store an input file and read it with the function `Input`. It should be straightforward, though:

In [16]:
def isvalid(line):
    words = line.split()
    return len(words) == len(set(words))

quantify(Input(4), isvalid)

337

**Part Two:**

In [17]:
def isvalid2(line):
    words = mapt(canon, line.split())
    return len(words) == len(set(words))

quantify(Input(4), isvalid2)

231

It took me less than five minutes, and my preparation with `Input` and `canon` helped, but I was still too slow to score any points--the top 20 took less than two minutes each.

# [Day 5](https://adventofcode.com/2017/day/5): A Maze of Twisty Trampolines, All Alike

Let's first make sure we can read the data/program okay:

In [18]:
program = mapt(int, Input(5))

program[:10]

(0, 2, 0, 0, -2, -2, -1, -4, -5, -6)

Now I'll make a little interpreter, `run`, which takes a program, loads it into memory,
 and executes the instruction, maintaining a program counter, `pc`, and doing the incrementing/branching as described in the puzzle,
until the program counter is out of range:

In [19]:
def run(program):
    memory = list(program)
    pc = steps = 0
    while pc in range(len(memory)):
        steps += 1
        oldpc = pc
        pc += memory[pc]
        memory[oldpc] += 1
    return steps
        
run(program)

364539

**Part Two:**

Part Two seems tricky, so I'll include an optional argument, `verbose`, and check if the printout it produces matches the example in the puzzle description:

In [20]:
def run2(program, verbose=False):
    memory = list(program)
    pc = steps = 0
    while pc in range(len(memory)):
        steps += 1
        oldpc = pc
        pc += memory[pc]
        memory[oldpc] += (-1 if memory[oldpc] >= 3 else 1)
        if verbose: print(steps, pc, memory)
    return steps
        
run2([0, 3, 0, 1, -3], True)

1 0 [1, 3, 0, 1, -3]
2 1 [2, 3, 0, 1, -3]
3 4 [2, 2, 0, 1, -3]
4 1 [2, 2, 0, 1, -2]
5 3 [2, 3, 0, 1, -2]
6 4 [2, 3, 0, 2, -2]
7 2 [2, 3, 0, 2, -1]
8 2 [2, 3, 1, 2, -1]
9 3 [2, 3, 2, 2, -1]
10 5 [2, 3, 2, 3, -1]


10

That looks right, so I can solve the puzzle:

In [21]:
run2(program)

27477714

Thanks to [Clement Sreeves](https://github.com/ClementSreeves) for the suggestion of making a distinction between the `program` and the `memory`. In my first version, `run` would mutate the argument, which was OK for a short exercise, but not best practice for a reliable API.

# [Day 6](https://adventofcode.com/2017/day/6): Memory Reallocation 

I had to read the puzzle description carefully, but then it is pretty clear what to do. I'll keep a set of previously seen configurations, which will all be tuples. But in the function `spread`, I want to mutate the configuration of banks, so I will convert to a list at the start, then convert back to a tuple at the end.

In [22]:
banks = vector('4	10	4	1	8	4	9	14	5	1	14	15	0	15	3	5')

def realloc(banks):
    "How many cycles until we reach a configuration we've seen before?"
    seen = {banks}
    for cycles in count_from(1):
        banks = spread(banks)
        if banks in seen:
            return cycles
        seen.add(banks)
        
def spread(banks):
    "Find the area with the most blocks, and spread them evenly to following areas."
    banks  = list(banks)
    maxi   = max(range(len(banks)), key=lambda i: banks[i])
    blocks = banks[maxi]
    banks[maxi] = 0
    for i in range(maxi + 1, maxi + 1 + blocks):
        banks[i % len(banks)] += 1
    return tuple(banks)

In [23]:
spread((0, 2, 7, 0))

(2, 4, 1, 2)

In [24]:
realloc((0, 2, 7, 0))

5

These tests look good; let's solve the problem:

In [25]:
realloc(banks)

12841

**Part Two:** Here I will just replace the `set` of `seen` banks with a `dict` of `{bank: cycle_number}`; everything else is the same, and the final result is the current cycle number minus the cycle number of the previously-seen tuple of banks.

In [26]:
def realloc2(banks):
    "When we hit a cycle, what is the length of the cycle?"
    seen = {banks: 0}
    for cycles in count_from(1):
        banks = spread(banks)
        if banks in seen:
            return cycles - seen[banks]
        seen[banks] = cycles

realloc2((0, 2, 7, 0))

4

In [27]:
realloc2(banks)

8038